In [42]:
import sys

import json
import pickle
import pandas
import torch
import asyncio
import numpy as np 
import pandas as pd
import seaborn as sns   
import torch.nn.functional as F
from matplotlib import pyplot as plt
sys.path.append("/home/v-oostapenko/dev/mttl")    
from scipy.stats import entropy as calc_entropy
from mttl.cluster_tuning.encodings import ClusterInfos
from mttl.datamodule.alpaca_data_module import AlpacaDataModule
from finetune_llama import Config
from mttl.cluster_tuning.cluster_reader import ClusterResult
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
device = "cuda" if torch.cuda.is_available() else "cpu"
# import matplotlib.pyplot as plt
# import seaborn as sns

import json
import os 
import sys 
import numpy as np
import click
import json
import tqdm
import copy
import torch
import datasets
from types import SimpleNamespace
sys.path.append("/home/v-oostapenko/dev/mttl")
from inst_follow.models.clm import CLM  
from transformers import LlamaTokenizer  
from mttl.models.poly import get_selector     
from mttl.models.modify_model import modify_transformer  
from finetune_llama import parse_config, Config
from inst_follow.utils import load_model, TopicRouter,disable_torch_init
from mttl.cluster_tuning.cluster_reader import ClusterResult
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import openai
openai.api_key = "f48b5a4f15dc4e58991738ab066ba465"

In [75]:
nc = 4
# loda cluster data
path_to_clusterers = f"/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/cbtm/clustered_{nc}_distances.json"
df = pd.read_json(path_to_clusterers, lines=True)
df["probs"]=df["cluster"].apply(lambda x: np.array(F.softmax(-torch.tensor(x) / 0.1, dim=-1).tolist()))
df["entropy"] = df["probs"].apply(lambda x: calc_entropy(x, axis=-1)/ np.log2(len(x)))
# convert to dict, with key being sp_id
hash_to_cluster = df.set_index("sp_id").to_dict()["cluster"]
# probs = np.array(df["probs"].tolist())

In [71]:
entr = df["entropy"].values
low_entr_idxs = np.where(entr <= 0.1)[0]
high_entr_idxs = np.where(entr > 0.4)[0]

In [76]:
from mttl.dataloader.alpaca_dataset_readers import AlpacaDataset               
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf", add_eos_token=True)
tokenizer.pad_token_id = 0 
max_input_length=256   
max_output_length=128 
data_dir=None          
dataset = AlpacaDataset(tokenizer, max_input_length, max_output_length, data_dir, train_on_inputs=False)    

100%|██████████| 1/1 [00:00<00:00, 655.87it/s]


In [ ]:
# rng = np.random.default_rng(42)
# all_test_idxs = rng.choice(len(dataset), size=1000, replace=False)

In [79]:
nc = 4
for nc in [4,8,16,32]:
    # loda cluster data
    path_to_clusterers = f"/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/cbtm/clustered_{nc}_distances.json"
    df = pd.read_json(path_to_clusterers, lines=True)
    df["probs"]=df["cluster"].apply(lambda x: np.array(F.softmax(-torch.tensor(x) / 0.1, dim=-1).tolist()))
    df["entropy"] = df["probs"].apply(lambda x: calc_entropy(x, axis=-1)/ np.log2(len(x)))
    # convert to dict, with key being sp_id
    hash_to_cluster = df.set_index("sp_id").to_dict()["cluster"]
    # probs = np.array(df["probs"].tolist())
    cluster_infos_new = ClusterInfos()     
    for i,(example, ex_dict_raw) in tqdm.tqdm(enumerate(zip(dataset,dataset.dataset)), total=len(dataset)):
        # topics_example = example["skill_set"]
        assert example.input_text.split("\n### Instruction:")[1].split("###")[0].strip() == ex_dict_raw["instruction"].strip(), (example.input_text.split("Instruction:")[1].split("###")[0].strip(), ex_dict_raw["instruction"])
        hash = example.hash     
        probs = hash_to_cluster[hash]
        is_test = 0
        # assert np.sum(probs)==1
        # print(np.sum(probs))    
        main_t = np.argmin(probs)    
        cluster_infos_new.is_test.extend([is_test])     
        cluster_infos_new.task_names.extend([str(main_t)])
        cluster_infos_new.cluster_ids.extend([int(main_t)])    
        cluster_infos_new.hashes.extend([hash])
        cluster_infos_new.cluster_dists.extend([probs])
    # assert sum(cluster_infos_new.is_test) == len(all_test_idxs)
    cluster_infos_new.save(f"/home/v-oostapenko/dev/mttl/inst_follow/cluster_infos/cbtm/cbtm_tfidf_kmeans_{nc}.pkl")

100%|██████████| 51760/51760 [01:39<00:00, 518.08it/s]


Bad pipe message: %s [b'Kk\x8a\x95\x9d\xf3y\x95\xfd\xc86x\xc4\x01W\xae\xfb\x97 \xa2\x9c\xfe\xc4C\x1dSU\xd4k\xf1KA,\x87\x19\xa1\xd7\x9e\\\xc4k\xc0N7\x12\xff\xfaN\x11\xbcV\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0', b'.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04']
Bad pipe message: %s [b'\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'8\xe90B0%\xd6\x99\x84q\x1d@\x08\x10\xfd\x8c#\x15 l\xc2X\x0ba\xa2s\xd5\x16>4,m\xd6t']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b'a7@\xaa7\xd2\xa8t\x8a \x9b\xfb\x82#T\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x0